# 2026 The Best Ski Trip Prediction

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

## Data Parsing and Cleaning

In [2]:
# Read the excel data
data_fp = list(Path().rglob('../data/2025*.xlsx'))[0]
excel_sheets = pd.read_excel(data_fp, sheet_name=None)
# Unpack the excel sheets
info_data, visitation_data, climate_data = [v for _, v in excel_sheets.items()]

### Parsing the info sheet 
1. Station ID and location
2. Winter ski weeks and dates

In [3]:
location_id = info_data.iloc[21:26,1].copy()
location_id = pd.DataFrame(location_id.str.split(" - ").to_list(), columns=["station_id", "location"])

st_id = (85291, 83085, 83024)
resort = ("Mt. Baw Baw", "Mt. Hotham", "Mt. Stirling")
addtl_rows = [{"station_id": id, "location": loc} for id, loc in zip(st_id, resort)]
location_id = pd.concat([location_id, pd.DataFrame(addtl_rows)]).reset_index(drop=True)
location_id["location"] = location_id["location"].str.replace(" AWS", "").str.replace(" SMHEA", "")
location_id.loc[location_id.station_id=="83024", "location"] = "Mt. Buller"
location_id.loc[location_id.station_id=="72161", "location"] = "Selwyn"
location_id = location_id.astype({"station_id": str})

print(location_id.info())

location_id

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   station_id  8 non-null      object
 1   location    8 non-null      object
dtypes: object(2)
memory usage: 260.0+ bytes
None


,station_id,location
0,71032,Thredbo
1,71075,Perisher
2,72161,Selwyn
3,83024,Mt. Buller
4,83084,Falls Creek
5,85291,Mt. Baw Baw
6,83085,Mt. Hotham
7,83024,Mt. Stirling


In [4]:
ski_season_dates = (info_data.iloc[35:, 1:].copy()
                    .rename(columns={"Unnamed: 1": "week", "Unnamed: 2": "dates"})
                    .reset_index(drop=True))
ski_season_dates["dates"] = ski_season_dates["dates"].dt.strftime("%d-%b")

print(ski_season_dates.info())

# Check the weeks
ski_season_dates.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   week    15 non-null     object
 1   dates   15 non-null     object
dtypes: object(2)
memory usage: 372.0+ bytes
None


,week,dates
0,Week 1,09-Jun
1,Week 2,16-Jun
2,Week 3,23-Jun
3,Week 4,30-Jun
4,Week 5,07-Jul


### Parsing the climate data

In [5]:
# Check unclean data
climate_data.tail()

,Bureau of Meteorology station number,Year,Month,Day,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres)
39808,72161,2025,7,24,3.0,-2.7,4.2
39809,72161,2025,7,25,6.0,-2.4,0.2
39810,72161,2025,7,26,2.7,0.0,10.2
39811,72161,2025,7,27,2.5,1.3,15.6
39812,72161,2025,7,28,2.2,-0.1,54.4


In [6]:
# Renaming columns using dictionary
cd_col_names = [col.lower() for col in climate_data.columns]
cd_col_names[0] = "station_id"
cd_col_names[-3:] = ["max_temp_c", "min_temp_c", "rainfall_mm"]
cd_col_names = {col_old: col_new for col_old, col_new in zip(climate_data.columns, cd_col_names)}
climate_data_clean = climate_data.copy().rename(columns=cd_col_names)

# Type formatting
climate_data_clean = climate_data_clean.astype({"station_id": str, "year": str})
climate_data_clean["date"] = pd.to_datetime(climate_data_clean.apply(lambda rec: f"{rec.day}/{rec.month}/{rec.year}", axis=1),
                                            format="%d/%m/%Y")

# Check datatype
print(climate_data_clean.info())

# Check records
climate_data_clean.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39813 entries, 0 to 39812
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   station_id   39813 non-null  object        
 1   year         39813 non-null  object        
 2   month        39813 non-null  int64         
 3   day          39813 non-null  int64         
 4   max_temp_c   38275 non-null  float64       
 5   min_temp_c   38280 non-null  float64       
 6   rainfall_mm  37857 non-null  float64       
 7   date         39813 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(2), object(2)
memory usage: 2.4+ MB
None


,station_id,year,month,day,max_temp_c,min_temp_c,rainfall_mm,date
39808,72161,2025,7,24,3.0,-2.7,4.2,2025-07-24
39809,72161,2025,7,25,6.0,-2.4,0.2,2025-07-25
39810,72161,2025,7,26,2.7,0.0,10.2,2025-07-26
39811,72161,2025,7,27,2.5,1.3,15.6,2025-07-27
39812,72161,2025,7,28,2.2,-0.1,54.4,2025-07-28


### Parsing the visitation data

In [7]:
# Converting to long format for easier processing
visitation_data_long = (visitation_data.copy()
                        .melt(id_vars=("Year", "Week"), var_name="location", value_name="visitors")
                        .rename(columns={"Year" : "year", "Week" : "week"}))
visitation_data_long = visitation_data_long.astype({"year": str, "week": str})
visitation_data_long.head()

,year,week,location,visitors
0,2014,1,Mt. Baw Baw,555
1,2014,2,Mt. Baw Baw,804
2,2014,3,Mt. Baw Baw,993
3,2014,4,Mt. Baw Baw,2976
4,2014,5,Mt. Baw Baw,11112


In [8]:
visitation_data_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1485 entries, 0 to 1484
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   year      1485 non-null   object
 1   week      1485 non-null   object
 2   location  1485 non-null   object
 3   visitors  1485 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 46.5+ KB


## Joining the Visitation Table with Location ID

In [9]:
visitation_location = pd.merge(visitation_data_long, location_id, on="location")
visitation_location.head()

,year,week,location,visitors,station_id
0,2014,1,Mt. Baw Baw,555,85291
1,2014,2,Mt. Baw Baw,804,85291
2,2014,3,Mt. Baw Baw,993,85291
3,2014,4,Mt. Baw Baw,2976,85291
4,2014,5,Mt. Baw Baw,11112,85291


In [10]:
# Build the ski-season calendar (Weeks 1–15) for every year we have in visitation
ski_mmdd = ["06-09","06-16","06-23","06-30","07-07","07-14","07-21","07-28",
            "08-04","08-11","08-18","08-25","09-01","09-08","09-15"]

def ski_calendar_for_year(y: int) -> pd.DataFrame:
    starts = pd.to_datetime([f"{y}-{md}" for md in ski_mmdd])
    cal = pd.DataFrame({"Year": y, "Week": range(1, 16), "Start": starts})
    cal["End"] = cal["Start"] + pd.Timedelta(days=7)
    return cal

years = sorted(np.append(visitation_data["Year"].unique(),[2025]))
ski_cal = pd.concat([ski_calendar_for_year(y) for y in years], ignore_index=True)

In [25]:
visitation_location

,year,week,location,visitors,station_id
0,2014,1,Mt. Baw Baw,555,85291
1,2014,2,Mt. Baw Baw,804,85291
2,2014,3,Mt. Baw Baw,993,85291
3,2014,4,Mt. Baw Baw,2976,85291
4,2014,5,Mt. Baw Baw,11112,85291
...,...,...,...,...,...
1315,2024,11,Perisher,49136,71075
1316,2024,12,Perisher,43382,71075
1317,2024,13,Perisher,23908,71075
1318,2024,14,Perisher,8244,71075


In [27]:
pd.merge(visitation_location, climate_data_clean, on=["station_id", "year"])

,year,week,location,visitors,station_id,month,day,max_temp_c,min_temp_c,rainfall_mm,date
0,2014,1,Mt. Baw Baw,555,85291,1,1,14.5,4.1,0.0,2014-01-01
1,2014,1,Mt. Baw Baw,555,85291,1,2,13.1,3.6,4.4,2014-01-02
2,2014,1,Mt. Baw Baw,555,85291,1,3,14.2,5.9,0.2,2014-01-03
3,2014,1,Mt. Baw Baw,555,85291,1,4,7.1,0.1,0.4,2014-01-04
4,2014,1,Mt. Baw Baw,555,85291,1,5,13.7,1.6,1.0,2014-01-05
...,...,...,...,...,...,...,...,...,...,...,...
482155,2024,15,Perisher,5336,71075,12,27,15.3,12.2,4.4,2024-12-27
482156,2024,15,Perisher,5336,71075,12,28,13.5,3.4,0.0,2024-12-28
482157,2024,15,Perisher,5336,71075,12,29,17.3,6.5,0.0,2024-12-29
482158,2024,15,Perisher,5336,71075,12,30,19.1,4.6,0.0,2024-12-30


In [ ]:
visitation_climate_2014_2024 = (pd.merge(visitation_location, climate_data_clean, on=["station_id", "year"])
                                .groupby(["location", "year", "week"])
                                .agg(visitors=("visitors")
                                     max_temp_c=("max_temp_c", "mean"),
                                     min_temp_c=("min_temp_c", "mean"),
                                     rainfall_mm=("rainfall_mm", "sum"))
                                .reset_index())

visitation_climate_2014_2024.head()

,location,year,week,max_temp_c,min_temp_c,rainfall_mm
0,Falls Creek,2014,1,10.381918,3.455495,1165.4
1,Falls Creek,2014,10,10.381918,3.455495,1165.4
2,Falls Creek,2014,11,10.381918,3.455495,1165.4
3,Falls Creek,2014,12,10.381918,3.455495,1165.4
4,Falls Creek,2014,13,10.381918,3.455495,1165.4


In [23]:
visitation_climate_2014_2024.to_csv(data_fp.parent / "merged/visitation_climate_2014_2024.csv", index=False)

## Exporting the data

In [9]:
location_id.to_csv(data_fp.parent / "location_id.csv", index=False)
location_id.to_parquet(data_fp.parent / "location_id.parquet", engine="fastparquet")

In [10]:
ski_season_dates.to_csv(data_fp.parent / "ski_season_dates.csv", index=False)
ski_season_dates.to_parquet(data_fp.parent / "ski_season_dates.parquet", engine="fastparquet")

In [11]:
climate_data_clean.to_csv(data_fp.parent / "climate_data.csv", index=False)
climate_data_clean.to_parquet(data_fp.parent / "climate_data.parquet", engine="fastparquet")

In [12]:
visitation_data.to_csv(data_fp.parent / "visitation_data_raw.csv", index=False)
visitation_data_long.to_csv(data_fp.parent / "visitation_data_long.csv", index=False)
visitation_data_long.to_parquet(data_fp.parent / "visitation_data_long.parquet", engine="fastparquet")

## Merging the tables

In [32]:
import pandas as pd
from pathlib import Path

# Build the ski-season calendar (Weeks 1–15) for every year we have in visitation
ski_mmdd = ["06-09","06-16","06-23","06-30","07-07","07-14","07-21","07-28",
            "08-04","08-11","08-18","08-25","09-01","09-08","09-15"]

def ski_calendar_for_year(y: int) -> pd.DataFrame:
    starts = pd.to_datetime([f"{y}-{md}" for md in ski_mmdd])
    cal = pd.DataFrame({"Year": y, "Week": range(1, 16), "Start": starts})
    cal["End"] = cal["Start"] + pd.Timedelta(days=7)
    return cal

years = sorted(np.append(visitation_data["Year"].unique(),[2025]))
ski_cal = pd.concat([ski_calendar_for_year(y) for y in years], ignore_index=True)

# Turn daily climate into ski-week climate
cl = climate_data.copy()
cl["Date"] = pd.to_datetime(dict(year=cl["Year"], month=cl["Month"], day=cl["Day"]), errors="coerce")
cl = cl.dropna(subset=["Date"]).sort_values(["Year","Date"]).reset_index(drop=True)

def tag_skiweek(df_year: pd.DataFrame) -> pd.DataFrame:
    # For each date, find the most recent ski-week start within the last 6 days
    cal = (ski_cal[ski_cal["Year"] == int(df_year["Year"].iloc[0])]
           [["Start","Week"]].sort_values("Start"))
    out = pd.merge_asof(
        df_year.sort_values("Date"),
        cal.rename(columns={"Start": "StartDate"}).sort_values("StartDate"),
        left_on="Date", right_on="StartDate",
        direction="backward", tolerance=pd.Timedelta(days=6)
    )
    return out

cl_tagged = pd.concat([tag_skiweek(g) for _, g in cl.groupby("Year")], ignore_index=True)
cl_tagged = cl_tagged.merge(ski_cal[["Year","Week","Start","End"]], on=["Year","Week"], how="left")
# Keep rows that truly fall inside a ski week window
cl_tagged = cl_tagged[(cl_tagged["Week"].notna()) & (cl_tagged["Date"] < cl_tagged["End"])].copy()
cl_tagged["Week"] = cl_tagged["Week"].astype(int)

# Aggregate to (Year, Week): temps = mean, rainfall = sum
weekly_climate = (
    cl_tagged.groupby(["Year","Week"], as_index=False)
    .agg({
        "Maximum temperature (Degree C)": "mean",
        "Minimum temperature (Degree C)": "mean",
        "Rainfall amount (millimetres)": "sum"
    })
)

# Clean visitation numbers
vis = visitation_data.copy()
for c in vis.columns:
    if c not in ["Year","Week"]:
        vis[c] = pd.to_numeric(vis[c], errors="coerce").fillna(0).clip(lower=0)

# Long format: one row per (Year, Week, Resort)
resort_cols = [c for c in vis.columns if c not in ["Year","Week"]]
vis_long = vis.melt(id_vars=["Year","Week"], value_vars=resort_cols,
                    var_name="Resort", value_name="Visitors")

# Add state for each resort
state_map = {
    "Mt. Stirling": "VIC",
    "Mt. Buller": "VIC",
    "Mt. Baw Baw": "VIC",
    "Mt. Hotham": "VIC",
    "Falls Creek": "VIC",
    "Selwyn": "NSW",
    "Thredbo": "NSW",
    "Perisher": "NSW",
    # Charlotte Pass will be created below and set to NSW
}
vis_long["State"] = vis_long["Resort"].map(state_map)

# Create Charlotte Pass (NSW) as the average of Perisher and Thredbo for each (Year, Week)
pt = vis_long[vis_long["Resort"].isin(["Perisher","Thredbo"])].pivot_table(
    index=["Year","Week"], columns="Resort", values="Visitors", aggfunc="mean"
).reset_index()

if {"Perisher","Thredbo"}.issubset(pt.columns):
    char = pd.DataFrame({
        "Year": pt["Year"],
        "Week": pt["Week"],
        "Resort": "Charlotte Pass",
        "Visitors": (pt["Perisher"] + pt["Thredbo"]) / 2.0,
        "State": "NSW"
    })
    vis_long = pd.concat([vis_long, char], ignore_index=True)

# Keep only ski-season weeks (1..15) to match the calendar
vis_long = vis_long[vis_long["Week"].between(1, 15)].reset_index(drop=True)

# Attach the weekly climate to every resort row
merged_data = vis_long.merge(weekly_climate, on=["Year","Week"], how="left")

# Save to data/merged_data.csv
out_path = Path("../data") / "merged"
out_path.parent.mkdir(parents=True, exist_ok=True)
merged_data.to_csv(out_path / "merged_data.csv", index=False)

print(f"Saved merged long-format data to {out_path.resolve()}")

# Rename
old_colnames = [col for col in merged_data.columns]
new_colnames = [col.lower() for col in old_colnames[0:5]] + ["max_temp_c", "min_temp_c", "rainfall_mm"]

merged_data = merged_data.rename(columns={old_col:new_col for old_col, new_col in zip(old_colnames, new_colnames)})

merged_data.head()

Saved merged long-format data to /Users/rapcanlas/Desktop/R2B2Datathon2025/R2B2/data/merged


,year,week,resort,visitors,state,max_temp_c,min_temp_c,rainfall_mm
0,2014,1,Mt. Baw Baw,555.0,VIC,5.022449,-0.338776,236.2
1,2014,2,Mt. Baw Baw,804.0,VIC,4.683673,-0.440816,125.4
2,2014,3,Mt. Baw Baw,993.0,VIC,0.308163,-2.553061,634.2
3,2014,4,Mt. Baw Baw,2976.0,VIC,1.236735,-2.363265,301.2
4,2014,5,Mt. Baw Baw,11112.0,VIC,-0.200000,-3.516327,257.0


In [30]:
# Aggregate 2025 weather data
climate_data_2025 = (climate_data_clean[climate_data_clean.year == "2025"].groupby(["station_id", "year","date"])
                     .agg(min_temp_c=("min_temp_c", "mean"),
                          max_temp_c=("max_temp_c", "mean"),
                          rainfall_mm=("rainfall_mm", "sum"))
                     .reset_index())
# Assign resort names
climate_data_2025 = climate_data_2025.merge(location_id, on="station_id").rename(columns={"location": "resort"})

ski_season_2025 = pd.to_datetime(ski_season_dates.dates + "-2025")

ski_season_2025 = pd.DataFrame({"week": np.arange(1,16), "week_start": ski_season_2025, "week_end": ski_season_2025 + pd.Timedelta(days=6)})

In [31]:
# Build interval index
intervals = pd.IntervalIndex.from_arrays(ski_season_2025["week_start"], ski_season_2025["week_end"], closed="both")

# Match each climate row to its week number
climate_data_2025["week"] = pd.cut(climate_data_2025.date, bins=intervals)
climate_data_2025["week"] = climate_data_2025["week"].cat.rename_categories(ski_season_2025.week.to_numpy())

In [ ]:
climate_data_winter_2025 = climate_data_2025.loc[~climate_data_2025.week.isna()]
climate_data_winter_2025.head()
climate_data_winter_2025 = climate_data_winter_2025[["year", "week", "resort", ""]]

,station_id,year,date,min_temp_c,max_temp_c,rainfall_mm,resort,week
159,71032,2025,2025-06-09,-5.2,-1.2,1.4,Thredbo,1
160,71032,2025,2025-06-10,-3.0,-1.4,0.0,Thredbo,1
161,71032,2025,2025-06-11,-4.4,-2.3,0.8,Thredbo,1
162,71032,2025,2025-06-12,-6.3,-0.9,0.0,Thredbo,1
163,71032,2025,2025-06-13,-6.0,-0.9,0.0,Thredbo,1


,station_id,year,date,min_temp_c,max_temp_c,rainfall_mm,resort,week
0,71032,2025,2025-01-01,7.2,17.6,0.0,Thredbo,NaN
1,71032,2025,2025-01-02,5.0,17.1,0.2,Thredbo,NaN
2,71032,2025,2025-01-03,5.0,18.0,0.0,Thredbo,NaN
3,71032,2025,2025-01-04,10.8,20.7,0.0,Thredbo,NaN
4,71032,2025,2025-01-05,13.1,21.6,0.0,Thredbo,NaN
...,...,...,...,...,...,...,...,...
1664,85291,2025,2025-07-24,-3.5,1.5,3.4,Mt. Baw Baw,7
1665,85291,2025,2025-07-25,-1.2,3.9,0.0,Mt. Baw Baw,7
1666,85291,2025,2025-07-26,-1.3,1.8,0.2,Mt. Baw Baw,7
1667,85291,2025,2025-07-27,0.3,1.6,6.8,Mt. Baw Baw,7


In [215]:
climate_data_winter_2025.to_csv(data_fp.parent / "climate_data_2025.csv", index=False)
climate_data_winter_2025.to_parquet(data_fp.parent / "climate_data_2025.parquet", engine="fastparquet")